# hw #2

## 1)

### (a)

#### (i) <br>
f '(x) = $e^x$ which is increasing in x $\in$ and R is a convex set. <br>
Therefore, f(x) = $e^x - 1$ is strictly convex

#### (ii) <br>
$H = \begin{pmatrix} \mathbf{0} & \mathbf{1} \\ \mathbf{1} & \mathbf{0} \end{pmatrix} $ <br>
Let $x = (x_1, x_2)^T$ <br>
Then $x^THx = (x_1, x_2) \begin{pmatrix} \mathbf{0}&\mathbf{1}\\ \mathbf{1}&\mathbf{0}\end{pmatrix} \begin{pmatrix}\mathbf{x_1}\\ \mathbf{x_2}\end{pmatrix} = 2x_1x_2 > 0$ since $x_1$, $x_2 \in R_{++}^2$ <br> Also, $R_{++}^2$ is a convex set. <br>
Therefore,$f(x_1, x_2) = x_1x_2$ is strictly convex.

#### (iii) <br>
$H = \alpha(1-\alpha)\begin{pmatrix}\mathbf{-x_1^{\alpha-2}x_2^{1-\alpha}}&\mathbf{x_1^{\alpha-1}x_2^{-\alpha}} \\ \mathbf{x_1^{\alpha-1}x_2^{-\alpha}}&\mathbf{-x_1^{\alpha}x_2^{-\alpha-1}} \end{pmatrix}$ <br>
Let $y = (y_1, y_2)^T$<br>
Then $y^THy = (y_1, y_2)*\alpha(1-\alpha)\begin{pmatrix}\mathbf{-x_1^{\alpha-2}x_2^{1-\alpha}}&\mathbf{x_1^{\alpha-1}x_2^{-\alpha}} \\ \mathbf{x_1^{\alpha-1}x_2^{-\alpha}}&\mathbf{-x_1^{\alpha}x_2^{-\alpha-1}} \end{pmatrix} \begin{pmatrix}\mathbf{y_1} \\ \mathbf{y_2}\end{pmatrix}$<br>
$= \alpha(1-\alpha)x_1^{\alpha-1}x_2^{-\alpha}[-y_1^2 \frac{x_2}{x_1}-y_2^2 \frac{x_1}{x_2}+2y_1y_2] <= \alpha(1-\alpha)x_1^{\alpha-1}x_2^{-\alpha}[-2\sqrt{y_1^2y_2^2}+2y_1y_2] <= 0$<br> Since $x_1, x_2 \in R_{++}^2$ and $0\leq \alpha \leq 1$ and equality holds iff $x_1 = x_2$ <br> Therefore, $f(x_1, x_2) = x_1^{\alpha}x_2^{1-\alpha}$ is positive semi-definite.

### (b) <br>
#### (i) <br>
$J(\theta) = \frac{1}{2}||A^T\theta - y||^2 = \frac{1}{2}(A^T\theta - y)^T(A^T\theta - y) = \frac{1}{2}\theta^TAA^T\theta - 2y^TA^T\theta + y^Ty$

$\nabla_{\theta} J(\theta) = AA^T\theta - Ay$<br>
The update rule: <br>
$\theta^{t+1} = \theta^{t} - \alpha (AA^T\theta - Ay)$

### (ii) <br>
According to HW#1 Problem(4), $J(\theta)$ is convex. <br>
Also, according to property 6: J is convex, continuous differetiable, then $x*$ is a global minimizer iff $\nabla_{\theta}J(\theta) = 0$ <br>
Thus, we have $\nabla_{\theta} J(\theta) = AA^T\theta - 2Ay = 0$ <br>
$\theta = (AA^T)^{-1}Ay$

## 2)

## 3)

### (a)

In [126]:
class myNaiveBayesClassifier:
    """
    Naive Bayes classifier for spam classification
    Using the multinomial event model and Laplace smoothing
    """
    def __init__(self):
        self.words_with_labels = []
        self.prior = []
        
    def get_bag_of_words_model(self):
        return self.words_with_labels
    
    def calc_log_prior(self, y_train):
        """
        calculate the prior probability of label y
        """
        numSpam = sum([ 1 for i in y_train if i== 1])
        PrSpam = float(numSpam) / len(y_train)
        PrNonSpam = 1 - PrSpam
        return [math.log(PrSpam), math.log(PrNonSpam)]
    
    def calc_conditional_prob_of_words(self, X_train,y_train):
        """
        calculate the conditional probability of word wj given label y
        """
        result = [[0]*len(X_train[0]), [0]*len(X_train[0])]

        for i in range(len(X_train)):
            for j in range(len(X_train[0])):
                if(y_train[i] == 1): # spam
                    result[0][j] += float(X_train[i][j])
                else: # non-spam
                    result[1][j] += float(X_train[i][j])
                    
        for j in range(len(X_train[0])):
            result[0][j] = (1.0 + result[0][j]) / (sum(np.add(result[0],1.0)))#+len(X_train[0]))
            result[1][j] = (1.0 + result[1][j]) / (sum(np.add(result[1],1.0)))#+len(X_train[0]))
        return result
    
    def calc_log_posterior(self, X_test):
        """
        calculate log of Pr(di|yi)
        """
        res = np.zeros((X_test.shape[0],2))

        for i in range(X_test.shape[0]):
            for j in range(len(X_test[0])):
                # probability the email is spam
                res[i][0] += X_test[i][j] * math.log(self.words_with_labels[0][j])
                # probability the email is non-spam
                res[i][1] += X_test[i][j] * math.log(self.words_with_labels[1][j])
        return res
                   
    def fit(self, X_train, y_train):
        """
        train the bag of words model
        """
        self.prior = self.calc_log_prior(y_train)
        self.words_with_labels = self.calc_conditional_prob_of_words(X_train, y_train)
    
    def predict(self, X_test):
        """
        predict whether given emails are spam or not
        """
        pred = [0] * X_test.shape[0]
        tmp = self.calc_log_posterior(X_test)
        
        for i in range(X_test.shape[0]):
            tmp[i] += tmp[i] + self.prior
            pred[i] = 1 if tmp[i][0] > tmp[i][1] else -1
        return pred
    
    def error_rate(self,y_pred, y_true):
        """
        calculate the error rate
        error_rate = # of wrongly classified documents in SPARSE.TEST / # of document in SPARSE.TEST * 100%
        """
        
        return float(sum([1 for i in range(len(y_pred))if y_pred[i] != y_true[i]])) / len(y_true) * 100
        

In [127]:
import scipy.io as sio
import math
import numpy as np

train = sio.loadmat('./spam_classification_matlab/train.mat')
test = sio.loadmat('./spam_classification_matlab/test.mat')

X_train = train['DENSE_TRAIN']
y_train = train['DENSE_Y_TRAIN']

X_test = test['DENSE_TEST_X']
y_true = test['DENSE_TEST_Y']


NB = myNaiveBayesClassifier()

NB.fit(X_train, y_train)

pred = NB.predict(X_test)
err = NB.error_rate(pred, y_true)
print("The error rate on the test set is " + str(err) + "%")

The error rate on the test set is 1.625%


### (b)

In [181]:
import queue as q
import pandas as pd

df = pd.read_csv("./spam_classification_matlab/TOKENS_LIST",sep='\s+', names=['index','value'])

dic = df['value']
# print(dic[:][2])

bag_of_words = NB.get_bag_of_words_model()

pq = q.PriorityQueue()

for i in range(len(bag_of_words[0])):
    tmp = (bag_of_words[0][i] / bag_of_words[1][i],i)
    pq.put(tmp)
    if pq.qsize() > 5:
        pq.get()

print("The five tokens that most indicative of the SPAM class are ")
lst = [-1] * 5
for i in range(pq.qsize()):
    lst[i] = pq.get()[1]
    print(dic[lst[i]])

The five tokens that most indicative of the SPAM class are 
websit
valet
unsubscrib
spam
httpaddr


## 4)

In [13]:
import numpy as np
import scipy.io as sio
import queue as q # priority queue
from scipy import stats

In [15]:
mnist_data = sio.loadmat('mnist_data.mat')

# training data
train = mnist_data['train']
X_train = train[0:,1:]
y_train = train[0:,0]

# testing data
test = mnist_data['test']
X_test = test[0:,1:]
y_test = test[0:,0]
print("successfully loading the training and testing data!")

In [67]:
class myKNNClassifier:
    def __init__(self, n_neighbors):
        """
        define number of classes
        """
        self.k_neighbors = n_neighbors
        
    def fit(self, X_train, y_train):
        """
        The training phase of the algorithm consists only
        of storing the feature vectors and class labels 
        of the training samples
        """
        self.X_train = X_train
        self.y_train = y_train
    
    def predict(self, X_test, lxnorm):
        """
        calculate the K nearest points based on different norm 
        e.g. l1-norm, l2-norm
    
        Input: 100 test images
        Output: the corresponding predicated label for each test image 
        
        Complexity: O( n*log(k) ) where n is the number of training
        examples and k is the number of nearest neighbours, multiplied
        by the number testing images, here p = 100.
        
        Using priority queue implementation
        """
        
        res = []
        # print(res)
        
        for i in range(X_test.shape[0]):
            ### find the k nearest points for each testing image ###
            norm = np.linalg.norm(X_test[i]-self.X_train,lxnorm,axis=1)
            mypq = q.PriorityQueue()
            for index, val in enumerate(norm):
                lst = [-val, index]
                mypq.put(lst)
                if(mypq.qsize() > self.k_neighbors):
                    mypq.get()
            tmp = []
            for i in range(mypq.qsize()):
                item = mypq.get()
                tmp.append(self.y_train[item[1]])
                
            res.append(stats.mode(tmp)[0][0])
            # print(res[i])
            
        return res
    
    def accuracy_score(self, y_pred, y_test):
        """
        return the classification accuracy
        accuracy = # correctly classified test images /  # total test images
        """
        return float(sum([1 for i in range(len(y_pred))if y_pred[i] == y_test[i]])) / len(y_test) * 100

### a)

In [68]:
# random sample 100 test images
sample_index = np.random.choice(test.shape[0],100)
X_test_sample = X_test[sample_index]
y_test_sample = y_test[sample_index]
print(sample_index[0:10])

[8376 6884 1414 5668 7397 4901  640 4649 2186 4491]


In [69]:
K = [1,5,9,13]
for k in K:
    # find the accuracy score for each k and find the best k value
    KNN = myKNNClassifier(k)
    KNN.fit(X_train, y_train)
    y_pred = KNN.predict(X_test_sample,2)# l2-norm

    acc = KNN.accuracy_score(y_pred, y_test_sample)
    print("When k = " + str(k) + " " + "the accuracy score is " + str(acc) + "%")

When k = 1 the accuracy score is 96.0%
When k = 5 the accuracy score is 97.0%
When k = 9 the accuracy score is 94.0%
When k = 13 the accuracy score is 95.0%


The above results showed that when k = 5 had the best performance.

### b)

In [70]:
K = [1,5,9,13]
for k in K:
    # find the accuracy score for each k and find the best k value
    KNN = myKNNClassifier(k)
    KNN.fit(X_train, y_train)
    y_pred = KNN.predict(X_test_sample,1) # using the l1-norm

    acc = KNN.accuracy_score(y_pred, y_test_sample)
    print("When k = " + str(k) + " " + "the accuracy score is " + str(acc) + "%")

When k = 1 the accuracy score is 95.0%
When k = 5 the accuracy score is 95.0%
When k = 9 the accuracy score is 92.0%
When k = 13 the accuracy score is 94.0%


The above results showed that when k = 1 or 5 had the best performance. I will choose the $l_2$ norm distance metric for MNIST since the accuracy score for $l_2$ norm is higher.